# Galaxy Pretraining
## Task:
Pick one of the four methods:
- Jigsaw (but you have to use ViTs): https://arxiv.org/abs/1603.09246Links to an external site.
- Masked Autoencoder: https://github.com/facebookresearch/maeLinks to an external site.
- BEiT: https://arxiv.org/pdf/2106.08254Links to an external site.
- MoCov3: https://github.com/facebookresearch/moco-v3Links to an external site.

Starting from random initialization, use the chosen pretraining method to train a vision backbone on the Galaxy datasetLinks to an external site. It is up to you to modify the method hyper-parameters, architecture, data-augmentations...

After pretraining is done, evaluate the quality of the learned representation via a linear probe on the frozen encoder. You have to report accuracy on the test set using a frozen backbone (so training of the backbone only uses the SSL pretraining loss, once training is done, you will freeze that backbone and train a linear probe on top of the learned representation using the dataset labels and report test accuracy). You can in addition to supervised finetuning (but frozen backbone results must be reported). 

You need to describe what you consider a reasonable accuracy and why, and try to reach that by playing with the different hyper-parameters. If thorough exploration was done and results are still subpar, it can be ok, but you need to strongly argue why that is the case and justify why you deem your exploration to be thorough enough.

## Submission:

- A 2-page (free-form) document detailing challenges, tips, tricks and your results along with any ablation you did
- The code (should be easy to install/execute for me so take care of all the data downloading etc as part of your script)

In [7]:
# python3 venv galaxy
# source galaxy/bin/activate

In [6]:
!pip install --upgrade pip
!pip install stable-pretraining torch lightning datasets polars pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.0/888.0 MB 38.2 MB/s  0:00:21m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 828.5/828.5 kB 9.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 10.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 36.7 MB/s  0:00:01m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 6.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 22.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 MB 36.8 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 832.4/832.4 kB 8.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 29.2 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14/14 [stable-pretraining]table-pretraining]


In [10]:
!pip install tqdm matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 23.6 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 17.8 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [matplotlib]6 [matplotlib]


In [9]:
from datasets import load_dataset

ds = load_dataset("matthieulel/galaxy10_decals")

/users/mspancho/Downloads/galaxy-pretraining-ssl/galaxy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 1774/1774 [00:08<00:00, 214.69 examples/s]
